### importing all the relevant libraries

In [433]:
import pandas as pd
import numpy as np
import kagglehub
from collections import Counter

### Declaring all the project constants

In [434]:
# Path of all the required datasets
DATASET_FILE_1 = 'aniruddhapa/swiggy-restaurants-dataset-of-metro-cities'
DATASET_FILE_2 = 'ashishjangra27/swiggy-restaurants-dataset'
DATASET_FILE_3 = 'rishikeshkonapure/zomato'
DATASET_FILE_4 = 'benroshan/online-food-delivery-preferencesbangalore-region'
DATASET_FILE_5 = 'saurabhbadole/zomato-delivery-operations-analytics-dataset'


### Reading all the dataset files required for cleaning and pre-processing

In [435]:
def read_all_files():
    try:
        df_datasets = {
            "DATASET_FILE_1": pd.read_csv(kagglehub.dataset_download(DATASET_FILE_1)+'/Swiggy_dataset.csv'),
            "DATASET_FILE_2": pd.read_csv(kagglehub.dataset_download(DATASET_FILE_2)+'/swiggy.csv').rename(columns={'cuisine':'cuisines'}),
            "DATASET_FILE_3": pd.read_csv(kagglehub.dataset_download(DATASET_FILE_3)+'/zomato.csv'),
            "DATASET_FILE_4": pd.read_csv(kagglehub.dataset_download(DATASET_FILE_4)+'/onlinedeliverydata.csv'),
            #"DATASET_FILE_5": pd.read_csv(kagglehub.dataset_download(DATASET_FILE_5)+'/Zomato Dataset.csv'),
        }
    except Exception as e:
        raise Exception(' while loading the datasets.\nError Msg:',e)
    else:
        print("All datasets have been loaded successfully!\nDataset Details:")
        for name, df in df_datasets.items():
            print(f"Size of {name} is {df.shape} and sample 5 rows:")
            print(df.sample(5), "\n")
    return df_datasets

### methods to clean the column data and store in one format

In [436]:
def remove_list_like_structure(cuisine_list,dataFrame):

    for cuisine in cuisine_list:
        if '[' in cuisine:
            temp_cuisine = cuisine[1:-1]
            temp_cuisine_list = []
            quote_idx = [-1,-1]
            count = 0
            for i in range(len(temp_cuisine)):
                if temp_cuisine[i] == "'" and count <2:
                    quote_idx[count] = i
                    count += 1
                if count >1:
                    temp_cuisine_list.append(temp_cuisine[quote_idx[0]+1:quote_idx[1]])
                    count =0
            dataFrame.replace(cuisine,', '.join(temp_cuisine_list),inplace=True)

def count_occurrence_of_each_cuisine(cuisine_list):
    
    cuisines = [cuisine.strip() for cuisines in cuisine_list for cuisine in cuisines.split(',')]
    return dict(Counter(cuisines))

def replace_cuisine_value_with_most_ordered(cuisine_list, cuisine_occurrence_count, cuisine_col_name, id_col_name):
    
    modified_cuisine_col_value = {}
    for _, row in cuisine_list.iterrows():
        cuisine_types = [c.strip() for c in sorted(row[cuisine_col_name].split(","))]
        most_ordered_cuisine = max(cuisine_types, key=lambda x: cuisine_occurrence_count.get(x, 0))
        modified_cuisine_col_value[row[id_col_name]] = most_ordered_cuisine
    return modified_cuisine_col_value

def split_city_area(city_entry):
    city_entry_org = city_entry.strip()
    for sep in [',', '&', '/']:
        parts = city_entry_org.split(sep)
        if len(parts) == 2:
            return pd.DataFrame({
                'city': [city_entry],
                'area': [parts[0].strip()],
                'new_city': [parts[1].strip()]
            })

def modify_rating_value(col_list):
    return {rate: rate.split('/')[0] for rate in col_list}

def replace_nan_values(dataFrame, column_name):
    value_counts = column_name.dropna().value_counts(normalize = True)
    return column_name.apply(lambda x: x if pd.notna(x) else np.random.choice(value_counts.index, p=value_counts.values))

## Data PreProcessing and Cleaning

#### Steps followed:
1. Dropping rows and columns (where ever needed) that are not required for data visualization.
2. Reassigning/refactoring the values of some columns so that they have the same format acorss each dataset. This will help data synchronising across all the datasets
3. Mergering all the individual pre-processed files into one and execute the step 1.
4. Filling the missing values in the final dataset based on the existing values of that particular column.
5. Returning the final clean dataset that will be used for data visualization.


In [437]:
def objective_1_preprocess_cleaning(df_datasets_map):
    try:
        for name, df in df_datasets_map.items():
            if name == 'DATASET_FILE_1':
                remove_list_like_structure(df['cuisines'],df)
                modified_cuisine_col_value = replace_cuisine_value_with_most_ordered(df[['id','cuisines']],count_occurrence_of_each_cuisine(df['cuisines']),'cuisines','id')
                df['cuisines'] = df['id'].map(modified_cuisine_col_value).fillna(df['cuisines'])

            elif name == 'DATASET_FILE_2':
                df.dropna(inplace=True)
                df_split_city_area = pd.concat([split_city_area(city) for city in df['city']], ignore_index=True)
                df_split_city_area.drop_duplicates(inplace=True)
                df = df.merge(df_split_city_area[['city', 'area', 'new_city']])
                modified_cuisine_col_value = replace_cuisine_value_with_most_ordered(df[['id','cuisines']],count_occurrence_of_each_cuisine(df['cuisines']),'cuisines','id')
                df['cuisines'] = df['id'].map(modified_cuisine_col_value).fillna(df['cuisines'])

            elif name == 'DATASET_FILE_3':
                df.dropna(inplace=True)
                df = df.assign(City='Bengaluru', id=range(1, len(df) + 1))
                df['rate'] = df['rate'].replace(modify_rating_value(df['rate']))
                # For Dish_liked column
                modified_dish_liked_col_value = replace_cuisine_value_with_most_ordered(df[['id','dish_liked']],count_occurrence_of_each_cuisine(df['dish_liked']),'dish_liked','id')
                df['dish_liked'] = df['id'].map(modified_dish_liked_col_value).fillna(df['dish_liked'])
                modified_cuisine_col_value = replace_cuisine_value_with_most_ordered(df[['id','cuisines']],count_occurrence_of_each_cuisine(df['cuisines']),'cuisines','id')
                df['cuisines'] = df['id'].map(modified_cuisine_col_value).fillna(df['cuisines'])
            
            elif name == 'DATASET_FILE_4':
                val = ['Zomato', 'Swiggy']
                for i in df['Medium (P1)']:
                    if str(i) == "Food delivery apps":
                        df['Medium (P1)'] = np.random.choice(val, size=len(df))
                #df.loc[df['Medium (P1)'] == "Food delivery apps", 'Medium (P1)'] = np.random.choice(['Zomato', 'Swiggy'], size=len(df))

            df_datasets_map[name] = df
    except Exception as e:
        raise Exception(" while cleaning and pre-processing the dataset.\nError Msg:",e)
    else:
        print("All the datasets has been successfully cleaned and pre-processed. Moving on to merger part...\n")
        dataset_after_preprocess_cleaned = merge_all_dataset(df_datasets_map)
    return dataset_after_preprocess_cleaned

def merge_all_dataset(df_datasets_map):
    try:
        final_dataset_list = []
        for name,df in df_datasets_map.items():
            df_temp = pd.DataFrame()
            if name == 'DATASET_FILE_1':
                current_dataset_cols = ['uuid','name','city','area','avgRating','totalRatingsString','cuisines','deliveryTime','veg']
                final_dataset_columns = ['Order_Id','Restaurant_Name','City','Area','Average_Rating','Total_Rating_String','Cuisine','Delivery_Time','Veg/Non-Veg']

            elif name == 'DATASET_FILE_2':
                current_dataset_cols = ['id','name','new_city','area','rating','rating_count','cuisines', 'cost']
                final_dataset_columns = ['Order_Id','Restaurant_Name','City','Area','Average_Rating','Total_Rating_String','Cuisine','Total_Order_Value']

            elif name == 'DATASET_FILE_3':
                current_dataset_cols = ['name','City','location','rate','votes','cuisines','dish_liked']
                final_dataset_columns = ['Restaurant_Name','City','Area','Average_Rating','Total_Rating_String','Cuisine', 'Dish_Liked']

            elif name == 'DATASET_FILE_4':
                current_dataset_cols = ['Age', 'Gender', 'Marital Status', 'Occupation', 'Monthly Income',  'Medium (P1)', 'Meal(P1)', 'Influence of rating', 'Freshness ', 'Temperature', 'Good Taste ', 'Good Quantity', 'Good Food quality', 'Poor Hygiene', 'Bad past experience', 'High Quality of package', 'Late Delivery', 'Long delivery time',  'Delay of delivery person getting assigned', 'Delay of delivery person picking up food', 'Maximum wait time', 'Less Delivery time', 'Wrong order delivered',  'Missing item', 'Order placed by mistake']
                final_dataset_columns = ['Age', 'Gender', 'Marital_Status', 'Occupation', 'Monthly_Income',  'App_Preference', 'Meal_Type', 'Influence_of_Rating', 'Freshness_of_Food', 'Temperature_of_Food', 'Taste_of_Food', 'Quantity_of_Food', 'Food_Quality', 'Poor_Hygiene', 'Bad_Past_Experience', 'High_Quality_of_Package', 'Late_Delivery', 'Long_Delivery_Time', 'Delay_of_Delivery_Person_Getting_Assigned', 'Delay_of_Delivery_Person_Picking_Up_Food', 'Maximum_Wait_Time', 'Less_Delivery_Time', 'Wrong_Order_Delivered',  'Missing_Item', 'Order_Placed_by_Mistake']
            
            df_temp[final_dataset_columns] = df[current_dataset_cols].copy()
            final_dataset_list.append(df_temp)
            
        dataset_after_preprocess_cleaned = pd.concat(final_dataset_list, ignore_index=True, sort=False)
        dataset_after_preprocess_cleaned = dataset_after_preprocess_cleaned[~dataset_after_preprocess_cleaned['Average_Rating'].str.contains('--',na=False)]
        dataset_after_preprocess_cleaned['Age'] = np.clip(np.random.normal(30, 8, len(dataset_after_preprocess_cleaned)), 18, 60).astype(int)
    except Exception as e:
        print(e)
        raise Exception(" while merging the dataset.\nError Msg:", e)
    else:
        print("All the datasets has been successfully Merged. Moving on to replace the missing values if any...\n")
        dataset_after_preprocess_cleaned = replace_missing_values(dataset_after_preprocess_cleaned)
    return dataset_after_preprocess_cleaned

def replace_missing_values(dataset_after_preprocess_cleaned):
    try:
        print("Count of Missing values\n",dataset_after_preprocess_cleaned.isna().sum(),'\n')
        for col in dataset_after_preprocess_cleaned.columns:
            if col not in ['Order_Id']:
                dataset_after_preprocess_cleaned[col] = replace_nan_values(dataset_after_preprocess_cleaned,dataset_after_preprocess_cleaned[col])
        dataset_after_preprocess_cleaned['Veg/Non-Veg'] = dataset_after_preprocess_cleaned['Veg/Non-Veg'].replace({True: 'Veg', False: "Non-Veg"})
    except Exception as e:
        raise Exception(" while replacing the missing values.\nError Msg:", e)
    else:
        print("Missing values has been successfully replaced maintaining original distribution percentage. Have a look at final dataset for visualization!\n")
        print(f"Size of final dataset file is {dataset_after_preprocess_cleaned.shape} and sample 5 rows:\n")
        print(dataset_after_preprocess_cleaned.sample(5), "\n")
        
    return dataset_after_preprocess_cleaned[['Order_Id','Age','Gender','Marital_Status','Occupation','Monthly_Income', 'App_Preference',	'Restaurant_Name'	,
        'City',	'Area',	'Cuisine',	'Veg/Non-Veg',	'Delivery_Time',	'Total_Order_Value',
      'Meal_Type',	'Dish_Liked',	'Average_Rating',	'Total_Rating_String', 	'Influence_of_Rating',
      'Freshness_of_Food', 'Temperature_of_Food',
       'Taste_of_Food', 'Quantity_of_Food', 'Food_Quality', 'Poor_Hygiene',
       'Bad_Past_Experience', 'High_Quality_of_Package', 'Late_Delivery',
       'Long_Delivery_Time', 'Delay_of_Delivery_Person_Getting_Assigned',
       'Delay_of_Delivery_Person_Picking_Up_Food', 'Maximum_Wait_Time',
       'Less_Delivery_Time', 'Wrong_Order_Delivered', 'Missing_Item',
       'Order_Placed_by_Mistake']]


In [438]:
# todo: raise expection has to be implemented
def __main__():
    try:
        df_datasets_map = read_all_files()
        dataset_after_preprocess_cleaned = objective_1_preprocess_cleaning(df_datasets_map)
    except Exception as e:
        print('Oops! Something went wrong', e)
    else:
        print('Woohoo!!! Program executed successfully.')

In [439]:
# configuring custom notebook settings
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',50)
pd.set_option('display.width',1000)
__main__()

All datasets have been loaded successfully!
Dataset Details:
Size of DATASET_FILE_1 is (8691, 18) and sample 5 rows:
     type      id                                               name                                  uuid       city           area avgRating totalRatingsString                                           cuisines costForTwoStrings  deliveryTime  minDeliveryTime  maxDeliveryTime                                            address       locality unserviceable    veg  City
1236    F  446793                   Kempe Gowda Biryani (Nati Style)  198394b6-e363-4786-ab95-b5d7c99399bb  Bangalore   Basavanagudi        --    Too Few Ratings                         ['Biryani' 'North Indian']      ₹500 FOR TWO            54               54               54  200 Rashtriya Vidyalaya Rd 2nd Block Jaya Naga...   Basavanagudi         False  False   NaN
1941    F  441144                          KONGU NATTU KARI VIRUNTHU  29689800-d6f9-483b-bce5-f425728f9971    Chennai      Annanagar       